In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output, State
import dash_table_experiments as dashTable

import plotly.graph_objs as go
import plotly

import numpy as np

import pandas as pd

import psycopg2

from sqlalchemy import *

import datetime as dt

from lifelines import *

import io

import flask

from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected = True)
import plotly.offline as offline

from sklearn import preprocessing


user = 'ghavranek65'
pw = 'an1965tho'
endpoint = 'lab-01-postgres.crac5qee4nrw.us-east-1.rds.amazonaws.com'
port = '5432'
db = 'Lab_01'
con_string = 'postgresql://{}:{}@{}:{}/{}'.format(user,pw,endpoint,port,db)

engine = create_engine(con_string)

connect = engine.connect()

from scipy.spatial.distance import mahalanobis

import scipy as sp

In [2]:
sql = '''
SELECT people.email, people.role, people.create_date, tmax.t_max_form, tmax.t_min_form FROM people 
LEFT JOIN
(SELECT activities.prospect, MAX(activities.date_time) AS t_max_form, MIN(activities.date_time) AS t_min_form FROM activities WHERE activities.date_time >= '2016-1-1' 
AND activities.act_type = 'Form' 
GROUP BY activities.prospect) AS tmax
ON people.email = tmax.prospect
WHERE people.create_date >= '2016-1-1'
'''
survivor = pd.read_sql(sql,con=connect)

In [3]:
survivor['create_date'] = pd.to_datetime(survivor['create_date']).dt.date
survivor['t_max_form'] = pd.to_datetime(survivor['t_max_form']).dt.date
survivor['t_min_form'] = pd.to_datetime(survivor['t_min_form']).dt.date

In [4]:
survivor['today_create'] = (dt.date(2017,10,17)-survivor['create_date']).dt.days  # used to determine if long enough to be observed
survivor['today_form'] = (dt.date(2017,10,17)-survivor['t_max_form']).dt.days  # used to determine if long enough to be observed

In [6]:
survivor['t_min_create'] = (survivor['t_min_form']-survivor['create_date']).dt.days # days between first activity and create date
survivor['t_max_min'] = (survivor.t_max_form - survivor.t_min_form).dt.days # days between first and last activity


<b>Create observed and duration fields</b>

In [7]:
# Find time limit when we can assume a prospect is dead.

# I need to find two things:  How long can a prospect exist before I can declare him dead AND How long do I wait after
# a prospect has been active before I can declare them dead.

# To answer the first question, let's take the time that the 95th percentile has been the system before they engaged.

survivor_form = survivor.dropna()

t_engage = int(survivor_form.t_min_create.quantile(.95)) # = time that 95% of the prospects have engaged if they were going to engage

# To answer the second question, lets take the time that 95% of the prospects who have engaged take between their first and last
# engagement

t_max_min_engage = int(survivor_form.t_max_min.quantile(.95))

In [8]:
def observed(today_create,t_max_form,t_max_min,create_date,today_form):
    if pd.isnull(t_max_form):
        if today_create > t_engage:
            return 1
        else: 
            return 0
    elif today_form > t_max_min_engage:
        return 1
    else:
        return 0

In [9]:
survivor['observed'] = survivor.apply(lambda x: observed(x['today_create'],x['t_max_form'],x['t_max_min'],x['create_date'],x['today_form']),axis=1)

In [10]:
def duration(t_max_form,observed,today_create,today_form,create_date):
    if pd.isnull(t_max_form):
        if observed == 1:
            return 0
        else:
            return today_create
    elif observed == 1:
        return (t_max_form - create_date).days
    else: 
        return today_create

In [11]:
survivor['duration'] = survivor.apply(lambda x: duration(x['t_max_form'],x['observed'],x['today_create'],x['today_form'],x['create_date']),axis=1)

In [25]:
T=survivor['duration']
E=survivor['observed']

kmf = KaplanMeierFitter()
survTable = kmf.fit(T,event_observed=E).survival_function_.reset_index()

In [26]:
survTable.head()

,timeline,KM_estimate
0,-8.0,1.0
1,-7.0,1.0
2,-6.0,1.0
3,-5.0,1.0
4,-2.0,1.0


In [30]:
trace = go.Scatter(x=survTable['timeline'],y=survTable['KM_estimate'])
layout = go.Layout(yaxis={'range':[0,1],'showline':False})
fig=go.Figure(data=[trace],layout=layout)
offline.iplot(fig)

Outlier discovery.  First, normalize the 